# Box Canyon Park 

https://www.google.com/maps/place/Box+Canyon+Park/@33.8857385,-117.7309815,910m/data=!3m2!1e3!4b1!4m6!3m5!1s0x80dcce2292288e59:0xc1684d99397ec327!8m2!3d33.8857385!4d-117.7284066!16s%2Fg%2F1tm8kwym?entry=ttu

## 1. Organize data

Create a folder under `semantic_SfM/data` and organize your data following the structures below. 

Using SfM products from Agisoft:
```
semantic_SfM/data
    ├── box_canyon_park
        ├── DJI_photos
        │       ├── DJI_0000.JPG
        │       ├── DJI_0001.JPG
        │       ├── ...
        │       └── DJI_0100.JPG
        ├── SfM_products
        │       ├── agisoft_cameras.xml
        │       ├── model.jpg
        │       ├── model.obj
        │       ├── model.mtl
        │       └── agisoft_model.las   
        ├── segmentations
        └── associations

```

## 2. Estimate memory usage

In [1]:
from ssfm.memory_calculator import memory_calculator

In [2]:
# pointcloud file
las_file = "../data/box_canyon_park/SfM_products/agisoft_model.las"
# image file sample
image_file = "../data/box_canyon_park/DJI_photos/DJI_0350.JPG"
# number of images
num_images = 129
# number of segmentation ids for each point in the point cloud
num_segmentation_ids = 5

memory_calculator(las_file, image_file, num_images, num_segmentation_ids)

+----------------------------------------+----------------------+
|              Memory Type               | Memory Required (GB) |
+----------------------------------------+----------------------+
|      Segmentation for each image       | 0.037181854248046875 |
| Pixel2point association for each image | 0.07436370849609375  |
| Point2pixel association for each image | 0.17169087380170822  |
|                                        |                      |
|      Segmentation for all images       |  4.796459197998047   |
| Pixel2point association for all images |  9.592918395996094   |
| Point2pixel association for all images |  22.14812272042036   |
|          pc_segmentation_ids           |  0.8584543690085411  |
|         pc_segmentation_probs          |  0.8584543690085411  |
|          keyimage_association          |   5.53703068010509   |
|                 Total                  |  43.79143973253667   |
+----------------------------------------+----------------------+


## 3. Create 2D Segmentation using SAM

In [1]:
from ssfm.image_segmentation import ImageSegmentation
import os

In [2]:
sam_params = {}
sam_params['model_name'] = 'sam'
sam_params['model_path'] = '../semantic_SfM/sam/sam_vit_h_4b8939.pth'
sam_params['model_type'] = 'vit_h'
sam_params['device'] = 'cuda:1'
sam_params['points_per_side'] = 64
sam_params['pred_iou_thresh'] = 0.96
sam_params['stability_score_thresh'] = 0.92

In [3]:
image_segmentor = ImageSegmentation(sam_params)   
image_segmentor.set_distortion_correction('../data/box_canyon_park/SfM_products/agisoft_cameras.xml')
image_folder_path = '../data/box_canyon_park/DJI_photos'
segmentation_folder_path = '../data/box_canyon_park/segmentations'
image_paths = [os.path.join(image_folder_path, f) for f in os.listdir(image_folder_path) if f.endswith('.JPG')]
image_segmentor.batch_predict(image_paths, segmentation_folder_path, maximum_size=1000, save_overlap=True)

Processing image 1/129.
Processing image 2/129.
Processing image 3/129.
Processing image 4/129.
Processing image 5/129.
Processing image 6/129.
Processing image 7/129.
Processing image 8/129.
Processing image 9/129.
Processing image 10/129.
Processing image 11/129.
Processing image 12/129.
Processing image 13/129.
Processing image 14/129.
Processing image 15/129.
Processing image 16/129.
Processing image 17/129.
Processing image 18/129.
Processing image 19/129.
Processing image 20/129.
Processing image 21/129.
Processing image 22/129.
Processing image 23/129.
Processing image 24/129.
Processing image 25/129.
Processing image 26/129.
Processing image 27/129.
Processing image 28/129.
Processing image 29/129.
Processing image 30/129.
Processing image 31/129.
Processing image 32/129.
Processing image 33/129.
Processing image 34/129.
Processing image 35/129.
Processing image 36/129.
Processing image 37/129.
Processing image 38/129.
Processing image 39/129.
Processing image 40/129.
Processin

## 4. Create projection associations

In [4]:
from ssfm.probabilistic_projection import *
import time

In [5]:
pointcloud_projector = PointcloudProjection(depth_filtering_threshold=0.2)
#pointcloud_projector = PointcloudProjection()

In [6]:

pointcloud_projector.read_camera_parameters('../data/box_canyon_park/SfM_products/agisoft_cameras.xml')
pointcloud_projector.read_mesh('../data/box_canyon_park/SfM_products/model.obj')
pointcloud_projector.read_pointcloud('../data/box_canyon_park/SfM_products/agisoft_model.las')


#pointcloud_projector.read_camera_parameters('../data/box_canyon_park/SfM_products/camera.json', '../data/box_canyon_park/SfM_products/shots.geojson')
#pointcloud_projector.read_pointcloud('../data/box_canyon_park/SfM_products/model.las')

In [7]:
# batch project
image_folder_path = '../data/box_canyon_park/DJI_photos'
save_folder_path = '../data/box_canyon_park/associations'

image_list = [f for f in os.listdir(image_folder_path) if f.endswith('.JPG')]
pointcloud_projector.parallel_batch_project_joblib(image_list, save_folder_path, num_workers=8)

Processing frames: 100%|██████████| 129/129 [06:21<00:00,  2.96s/it]


In [8]:
# build keyimage associations
t1 = time.time()
pointcloud_projector.build_associations_keyimage('../data/box_canyon_park/associations', '../data/box_canyon_park/segmentations')
t2 = time.time()
print('Time for building keyimage associations: ', t2 - t1)

Time for building keyimage associations:  103.16834735870361


## 5. Run object registration

In [9]:
from ssfm.object_registration import *
from ssfm.post_processing import PostProcessing

In [ ]:
# Set paths

#time.sleep(3600)
pointcloud_path = '../data/box_canyon_park/SfM_products/agisoft_model.las'
segmentation_folder_path = '../data/box_canyon_park/segmentations'
image_folder_path = '../data/box_canyon_park/DJI_photos'
association_folder_path = '../data/box_canyon_park/associations'

# Create object registration
t1 = time.time()
obr = ObjectRegistration(pointcloud_path, segmentation_folder_path, association_folder_path)
t2 = time.time()
print('Time elapsed for creating object registration: {}'.format(t2-t1))

# Run object registration
obr.object_registration(iou_threshold=0.5, save_semantics=True)
#obr.object_registration(iou_threshold=0.5)

# save semantic point cloud
obr.save_semantic_pointcloud('../data/box_canyon_park/associations/semantic_model.las')

In [ ]:
# shuffle semantic ids
semantic_pc_file_path = '../../data/box_canyon_park/semantic_model.las'
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids()
save_las_path = '../../data/box_canyon_park/semantic_model_shuffled.las'
post_processing.save_semantic_pointcloud(save_las_path)

In [ ]:

pointcloud_path = '../../data/box_canyon_park/SfM_products/agisoft_model.las'
segmentation_folder_path = '../../data/box_canyon_park/segmentations'
image_folder_path = '../../data/box_canyon_park/DJI_photos'
association_folder_path = '../../data/box_canyon_park/associations'

obr = ObjectRegistration(pointcloud_path, segmentation_folder_path, association_folder_path)

add_semantics_to_pointcloud_flag = True
if add_semantics_to_pointcloud_flag:
    image_id = 128
    semantics_folder_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}.npy'.format(image_id))
    save_las_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}_depth_filter.las'.format(image_id))
    add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path) 